<a href="https://colab.research.google.com/github/Ulugbek9403/ml_edu/blob/master/notebooks/04_Numpy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Numpy (Часть 3)


> 🚀 В этой практике нам понадобятся: `numpy==1.26.4`

> 🚀 Установить вы их можете с помощью команды: `%pip install numpy==1.26.4`


## Содержание

* [Broadcasting (трансляция)](#Broadcasting-трансляция)
* [Изменение размеров массива (Reshape)](#Изменение-размеров-массива-Reshape)
* [Объединение массивов](#Объединение-массивов)
  * [Задание - объединяем](#Задание---объединяем)
  * [Задание - расширяем](#Задание---расширяем)
* [Сортировка массива](#Сортировка-массива)
  * [Задание - покопаемся в доках](#Задание---покопаемся-в-доках)
  * [Задание - никуда без сортировки](#Задание---никуда-без-сортировки)
* [Задачки](#Задачки)
* [Полезные ссылки](#Полезные-ссылки)


В этом ноутбуке:
- Broadcasting - операции между массивами неодинаковых размеров
- Изменение формы массива  - reshape
- Объединение массивов - concatenate
- Сортировка по элементам и индексам - sort, argsort

Теперь рассмотрим некоторые "рутинные" процедуры работы с массивами, которые пригодятся в дальнейшем. За более полной документацией сюда https://numpy.org/doc/stable/reference/routines.array-manipulation.html

In [1]:
import numpy as np

## Broadcasting (трансляция)

Broadcasting - это механизм  выполнения математических операций с массивами *неодинаковых* размеров. Тема достаточно "широкая" =)

На деле механизм очень полезен и его понять его стоит, так как он является одним из мощных инструментов библиотеки numpy. Допустим, мы имеет вектор $(4,)$, матрицу $(3, 4)$ и хотим умножить этот вектор на каждую строку матрицы. Самый простой способ - пройти в цикле по строкам:

In [2]:
vec = np.arange(4)
mtrx = np.ones((3, 4))
print(vec)
print(mtrx)

[0 1 2 3]
[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]


In [3]:
result = np.empty_like(mtrx)

for i_row in range(mtrx.shape[0]):
    result[i_row, :] = mtrx[i_row, :] * vec

print(result)

[[0. 1. 2. 3.]
 [0. 1. 2. 3.]
 [0. 1. 2. 3.]]


С другой стороны, можно склонировать вектор по размеру матрицы, чтобы затем просто перемножить поэлементно две матрицы.

In [4]:
# Функция np.tile() повторяет массив то количество раз и по тем осям,
#   которые заданы в аргументе reps
vec_m = np.tile(vec, reps=(3, 1))
print(vec_m)
print(vec_m * mtrx)

[[0 1 2 3]
 [0 1 2 3]
 [0 1 2 3]]
[[0. 1. 2. 3.]
 [0. 1. 2. 3.]
 [0. 1. 2. 3.]]


А можно вспомнить про броадкастинг и просто умножить вектор на матрицу:

In [5]:
# Умножение, используя броадкастинг
print(vec * mtrx)

[[0. 1. 2. 3.]
 [0. 1. 2. 3.]
 [0. 1. 2. 3.]]


Таким образом, броадкастинг - это механизм распространения элементов операций по разметностям, по которым количество элементов не соотносится с другим операндом. В последней ячейке вектор был продублирован на каждую строку так, что размерности соотносятся.

> Обязательно прочитайте примеры броадкастинга в [статье](https://numpy.org/doc/stable/user/basics.broadcasting.html).

Броадкастинг также работает и при умножении скаляра на массив:

In [6]:
print(2 * mtrx)

[[2. 2. 2. 2.]
 [2. 2. 2. 2.]
 [2. 2. 2. 2.]]


Чтобы больше понять правила броадкастинга, посмотрим на результаты операций 3D матриц:

In [7]:
x = np.ones((4, 1, 1))
y = np.random.randint(0, 5, size=(4, 3, 2))

print((x * y).shape)

(4, 3, 2)


In [8]:
x = np.ones((3, 1))
y = np.random.randint(0, 5, size=(4, 3, 2))

print((x * y).shape)

(4, 3, 2)


In [9]:
x = np.ones((1, 3, 2))
y = np.random.randint(0, 5, size=(5, 1, 2))

print((x * y).shape)

(5, 3, 2)


В последних примерах можно увидеть, что размерности должны быть соотносимы (количества размерностей равны). В случае неравенства количества размерностей новые размерности со значением 1 добавляются слева ($(3, 1) \rightarrow (1, 3, 1)$ в одном из примеров).

> Добавление размерности было и в случае умножения вектора на матрицу: $(4,)*(3, 4) \rightarrow (1, 4)*(3, 4)$

Броадкастинг делается по тем осям, которые у одного операнда имеют размерность 1 путем копирования.

> В нашем случае $(1, 4)*(3, 4) \rightarrow (3, 4)*(3, 4)$

Осознать броадкастинг сразу не совсем просто, но главное не забывать, что такой инструмент есть и им нужно пользоваться во избежание лишнего кода.

## Изменение размеров массива (Reshape)

Ещё одна интересная тема, которую стоит рассмотреть. Мы уже касались вопроса приведения массива к выпрямленному виду - это один из видов изменения размерности массива.

По сути, размерность массива - это порядок выстроенных элементы в нём. Изменение размерности - это изменение структуры **без изменения элементов или порядка**.

Рассмотрим на примере:

In [10]:
arr = np.arange(20)
print(arr)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


Вот мы имеем вектор из 20 элементов в порядке возрастания. Теперь, допустим, нам нужно сделать массив размером $(4, 5)$, в котором каждый ряд - это продолжение предыдущего по возрастанию чисел. Для начала сделаем руками:

In [11]:
mtrx = np.ndarray((4, 5), dtype=np.int32)
mtrx[0, :] = arr[:5]
mtrx[1, :] = arr[5:10]
mtrx[2, :] = arr[10:15]
mtrx[3, :] = arr[15:]

print(mtrx)

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]


Получилось как надо, но есть два недостатка:
- Если элементов будет больше и нужен будет другой размер - код не универсален;
- Приходится расставлять данные вручную.

Теперь взглянем, как работает метод `ndarray.reshape()`:

In [12]:
print(arr.reshape((4, 5)))

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]


В результате происходит тоже самое. То есть, элементы расположились в порядке размещения по размерностям: начиная с первой строки (нулевая размерность) заполняется по колонкам (первая размерность), а как только заполнение по колонкам закончено - переходим на следующий ряд.

Таким образом, в какую бы размерность не происходило изменение размера - расположение будет придерживаться этого правила.

Для лучшего понимания посмотрим на другие представления:

In [13]:
print(arr.reshape((2, 10)))
print(arr.reshape((5, 4)))
print(arr.reshape((20, 1)))

[[ 0  1  2  3  4  5  6  7  8  9]
 [10 11 12 13 14 15 16 17 18 19]]
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]
 [16 17 18 19]]
[[ 0]
 [ 1]
 [ 2]
 [ 3]
 [ 4]
 [ 5]
 [ 6]
 [ 7]
 [ 8]
 [ 9]
 [10]
 [11]
 [12]
 [13]
 [14]
 [15]
 [16]
 [17]
 [18]
 [19]]


Таким образом, можно приводить к любой размерности с учетом правила:
> Количество элементов должно сохраняться

Но бывают ситуации, когда мы точно не знаем количество элементов, но нам нужно точно создать, например, 4 строки в матрице. В этом поможет неопределенная размерность, которая задается числом -1:

In [14]:
print(arr.reshape((4, -1)))

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]
 [15 16 17 18 19]]


Таким образом можно задать, что матрица должна быть 2D с фиксированным количеством строк, но без фиксации на количество столбцов. Конечно же, если в исходном будет 21 элемент, то такой трюк не сработает, потому что 21 не делится на 4 без остатка.

## Объединение массивов

При работе с массивами бывают случаи, когда несколько массивов необходимо объединить в один. Для этого поможет функция `np.concatenate()` и понимание ее основных правил.

In [15]:
x = np.arange(10).reshape((2, 5))
y = np.arange(15).reshape((3, 5))

print(x)
print(y)

[[0 1 2 3 4]
 [5 6 7 8 9]]
[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]]


Для объединения массивов необходимо, чтобы одна из размерностей имела одинаковое количество элементов. Это достаточно логичное требование задаётся аргументом `axis` в функции. Для запоминания направления линии соединения и описыавется числом - "0"-строка, "1" - столбец.

In [16]:
print(np.concatenate((x, y), axis=0))

[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]]


Обратите внимание, что массивы передаются в виде кортежа, что позволяет объединять два и более массивов.

### Задание - объединяем

Объедините три массива вдоль колонок:

In [18]:
x = np.arange(9).reshape((3, 3))
y = np.arange(12).reshape((3, 4))
z = np.arange(15).reshape((3, 5))

print(x)
print(y)
print(z)

import numpy as np

x = np.arange(9).reshape((3, 3))
y = np.arange(12).reshape((3, 4))
z = np.arange(15).reshape((3, 5))

print("Матрица x:\n", x)
print("Матрица y:\n", y)
print("Матрица z:\n", z)

combined_array = np.concatenate((x, y, z), axis=1)

print("Объединенная матрица:\n", combined_array)


[[0 1 2]
 [3 4 5]
 [6 7 8]]
[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]]
Матрица x:
 [[0 1 2]
 [3 4 5]
 [6 7 8]]
Матрица y:
 [[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]]
Матрица z:
 [[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]]
Объединенная матрица:
 [[ 0  1  2  0  1  2  3  0  1  2  3  4]
 [ 3  4  5  4  5  6  7  5  6  7  8  9]
 [ 6  7  8  8  9 10 11 10 11 12 13 14]]


### Задание - расширяем

Напишите реализацию добавления стобца единиц к матрице слева:

In [20]:
x = np.arange(1, 10).reshape((3, 3))
print(x)

import numpy as np

x = np.arange(1, 10).reshape((3, 3))
print("Исходная матрица:\n", x)

x_with_ones = np.c_[np.ones(x.shape[0]), x]

print("Матрица с добавленным столбцом единиц:\n", x_with_ones)


[[1 2 3]
 [4 5 6]
 [7 8 9]]
Исходная матрица:
 [[1 2 3]
 [4 5 6]
 [7 8 9]]
Матрица с добавленным столбцом единиц:
 [[1. 1. 2. 3.]
 [1. 4. 5. 6.]
 [1. 7. 8. 9.]]


## Сортировка массива

Numpy как серьёзная библиотека работы с массивами имеет также и функционал сортировки. Для этого есть функция `np.sort()`, которая производит сортировку элементов и возвращает отсортированный массив:

In [21]:
unsorted_arr = np.array([2, 4, 5, 1, 2, 7, 5, 0, -1, 3, 5])
sorted_arr = np.sort(unsorted_arr)

print(unsorted_arr)
print(sorted_arr)

[ 2  4  5  1  2  7  5  0 -1  3  5]
[-1  0  1  2  2  3  4  5  5  5  7]


Помимо явной сортировки существует также функция `np.argsort()`, которая возвращает не отсортированный массив, а массив индексов исходного массива в порядке, который даёт отсортированный исходный. Посмотрим:

In [22]:
sorted_indices = np.argsort(unsorted_arr)
print(sorted_indices)

# А теперь воспользуемся индексами и отобразим исходный массив с индексацией
print(unsorted_arr[sorted_indices])

[ 8  7  3  0  4  9  1  2  6 10  5]
[-1  0  1  2  2  3  4  5  5  5  7]


### Задание - покопаемся в доках

По такому же принципу именования найдите функцию в документации, которая возвращает индекс максимального/минимального (в завимости от варианта) элемента в массиве.

Воспользуйтесь найденной функцией и найдите индексы максимальных элементов в каждом столбце 2D матрицы:

In [26]:
arr = np.array([
    [11, -1, 1, 3, 5],
    [6, 1, 0, -1, 10],
    [1, 3, 2, 3, 6],
    [5, 2, 7, 8, 9],
])

print(arr)

import numpy as np

arr = np.array([
    [11, -1, 1, 3, 5],
    [6, 1, 0, -1, 10],
    [1, 3, 2, 3, 6],
    [5, 2, 7, 8, 9],
])

print(arr)

max_indices = np.argmax(arr, axis=0)

print("Индексы максимальных элементов в каждом столбце:", max_indices)
min_indices = np.argmin(arr, axis=0)


[[11 -1  1  3  5]
 [ 6  1  0 -1 10]
 [ 1  3  2  3  6]
 [ 5  2  7  8  9]]
[[11 -1  1  3  5]
 [ 6  1  0 -1 10]
 [ 1  3  2  3  6]
 [ 5  2  7  8  9]]
Индексы максимальных элементов в каждом столбце: [0 2 3 3 1]


### Задание - никуда без сортировки

Напишите код сортировки массива по столбцам:

In [28]:
arr = np.array([
    [11, 2, 1, 3, 5],
    [6, 2, 0, -1, 10],
    [1, 3, 2, 3, 5],
    [5, 2, 7, 8, 9],
])
print(arr)

import numpy as np

arr = np.array([
    [11, 2, 1, 3, 5],
    [6, 2, 0, -1, 10],
    [1, 3, 2, 3, 5],
    [5, 2, 7, 8, 9],
])
print("Исходный массив:\n", arr)

sorted_arr = np.sort(arr, axis=0)

print("Отсортированный по столбцам массив:\n", sorted_arr)


[[11  2  1  3  5]
 [ 6  2  0 -1 10]
 [ 1  3  2  3  5]
 [ 5  2  7  8  9]]
Исходный массив:
 [[11  2  1  3  5]
 [ 6  2  0 -1 10]
 [ 1  3  2  3  5]
 [ 5  2  7  8  9]]
Отсортированный по столбцам массив:
 [[ 1  2  0 -1  5]
 [ 5  2  1  3  5]
 [ 6  2  2  3  9]
 [11  3  7  8 10]]


## Задачки

Создайте массив, состоящий из случайных элементов в диапазоне $[-10; 20]$ размером $(5, 7)$:

In [30]:
import numpy as np

random_array = np.random.randint(low=-10, high=21, size=(5, 7))

print(random_array)


[[ -7  11  -4  -6  19  -4  16]
 [ 11   4   9   6 -10  -3  12]
 [ 12   6 -10   3  -7  16  11]
 [  5  -1   8  18  -2  -4  17]
 [ 17  10  10  -5 -10   7  10]]


Добавьте границу в виде нулей со всех сторон 2D массива:

In [32]:
data = np.arange(1, 16).reshape((3, 5))
print(data)

import numpy as np

data = np.arange(1, 16).reshape((3, 5))
print("Исходный массив:\n", data)

data_with_border = np.pad(data, pad_width=1, mode='constant', constant_values=0)

print("Массив с границей:\n", data_with_border)


[[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]]
Исходный массив:
 [[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]]
Массив с границей:
 [[ 0  0  0  0  0  0  0]
 [ 0  1  2  3  4  5  0]
 [ 0  6  7  8  9 10  0]
 [ 0 11 12 13 14 15  0]
 [ 0  0  0  0  0  0  0]]


Разберитесь с функцией `np.clip()` и произведите ограничение массива, чтобы в нем значения были не более шести и не менее двух:

In [59]:
data = np.arange(1, 16).reshape((3, 5))
print(data)
import numpy as np

data = np.arange(1, 16).reshape((3, 5))
print("Исходный массив:\n", data)

clipped_data = np.clip(data, a_min=2, a_max=6)

print("Массив с ограниченными значениями:\n", clipped_data)



[[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]]
Исходный массив:
 [[ 1  2  3  4  5]
 [ 6  7  8  9 10]
 [11 12 13 14 15]]
Массив с ограниченными значениями:
 [[2 2 3 4 5]
 [6 6 6 6 6]
 [6 6 6 6 6]]


Произведите модификацию массива так, чтобы значение в колонке результата соответствовало исходному значению, возведенному в степень индекса колонки:

In [40]:
data = np.full((3, 5), fill_value=2, dtype=np.int32)
print(data)

import numpy as np

data = np.full((3, 5), fill_value=1, dtype=np.int32)
print("Исходный массив:\n", data)


for i in range(data.shape[1]):
    data[:, i] = data[:, i] * i

print("Модифицированный массив:\n", data)



[[2 2 2 2 2]
 [2 2 2 2 2]
 [2 2 2 2 2]]
Исходный массив:
 [[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]]
Модифицированный массив:
 [[0 1 2 3 4]
 [0 1 2 3 4]
 [0 1 2 3 4]]


Умножьте каждую колонку на её индекс:

In [42]:
data = np.full((3, 5), fill_value=2, dtype=np.int32)
print(data)

import numpy as np

data = np.full((3, 5), fill_value=2, dtype=np.int32)
print("Исходный массив:\n", data)

for i in range(data.shape[1]):
    data[:, i] *= i

print("Модифицированный массив:\n", data)


[[2 2 2 2 2]
 [2 2 2 2 2]
 [2 2 2 2 2]]
Исходный массив:
 [[2 2 2 2 2]
 [2 2 2 2 2]
 [2 2 2 2 2]]
Модифицированный массив:
 [[0 2 4 6 8]
 [0 2 4 6 8]
 [0 2 4 6 8]]


Умножьте каждый чётный ряд в матрице на вектор:

In [58]:
data = np.repeat(np.arange(1, 8), 4).reshape(7, 4)
vec = np.array([1, 2, 1, 2])
print(data)
import numpy as np

data = np.repeat(np.arange(1, 8), 4).reshape(7, 4)
vec = np.array([1, 2, 1, 2])
print("Исходная матрица:\n", data)

for i in range(data.shape[0]):
    if i % 2 == 0:
        data[i] *= vec

print("Модифицированная матрица:\n", data)



[[1 1 1 1]
 [2 2 2 2]
 [3 3 3 3]
 [4 4 4 4]
 [5 5 5 5]
 [6 6 6 6]
 [7 7 7 7]]
Исходная матрица:
 [[1 1 1 1]
 [2 2 2 2]
 [3 3 3 3]
 [4 4 4 4]
 [5 5 5 5]
 [6 6 6 6]
 [7 7 7 7]]
Модифицированная матрица:
 [[ 1  2  1  2]
 [ 2  2  2  2]
 [ 3  6  3  6]
 [ 4  4  4  4]
 [ 5 10  5 10]
 [ 6  6  6  6]
 [ 7 14  7 14]]


Определите индексы двух наибольших элементов в матрице по каждому ряду:

In [57]:
data = np.array([[20, 22,  2, 14, 25,  8],
                [ 7, 25, 23,  3, 22, 15],
                [ 8, 16,  9, 22,  0,  9],
                [ 4, 24, 24, 28,  3, 40]])
print(data)
import numpy as np

data = np.array([[20, 22, 2, 14, 25, 8],
                [ 7, 25, 23, 3, 22, 15],
                [ 8, 16, 9, 22, 0, 9],
                [ 4, 24, 24, 28, 3, 40]])
print("Исходная матрица:\n", data)

result = np.zeros((data.shape[0], 2), dtype=int)
for i in range(data.shape[0]):
    sorted_indices = np.argsort(data[i])
    result[i] = sorted_indices[-2:][::-1]

print("Индексы двух наибольших элементов по каждому ряду:\n", result)




[[20 22  2 14 25  8]
 [ 7 25 23  3 22 15]
 [ 8 16  9 22  0  9]
 [ 4 24 24 28  3 40]]
Исходная матрица:
 [[20 22  2 14 25  8]
 [ 7 25 23  3 22 15]
 [ 8 16  9 22  0  9]
 [ 4 24 24 28  3 40]]
Индексы двух наибольших элементов по каждому ряду:
 [[4 1]
 [1 2]
 [3 1]
 [5 3]]


Выберите две случайные колонки из массива:

<details><summary>Подсказка 1</summary>
Для случайного выбора полезно воспользоваться функцией `np.random.choice()`
</details>

<details>
<summary>Подсказка 2</summary>
`np.random.choice()` принимает на вход 1D массив, так что для выбора можно создать массив индексов колонок (`range()` с количеством колонок) и из него функция выберет два случайных индекса
</details>

In [48]:
data = np.array([[20, 22,  2, 14, 25,  8],
                [ 7, 25, 23,  3, 22, 15],
                [ 8, 16,  9, 22,  0,  9],
                [ 4, 24, 24, 28,  3, 40]])
print(data)
import numpy as np

data = np.array([[20, 22, 2, 14, 25, 8],
                [ 7, 25, 23, 3, 22, 15],
                [ 8, 16, 9, 22, 0, 9],
                [ 4, 24, 24, 28, 3, 40]])
print("Исходная матрица:\n", data)

column_indices = np.random.choice(data.shape[1], size=2, replace=False)
result = data[:, column_indices]

print("Случайно выбранные две колонки:\n", result)


[[20 22  2 14 25  8]
 [ 7 25 23  3 22 15]
 [ 8 16  9 22  0  9]
 [ 4 24 24 28  3 40]]
Исходная матрица:
 [[20 22  2 14 25  8]
 [ 7 25 23  3 22 15]
 [ 8 16  9 22  0  9]
 [ 4 24 24 28  3 40]]
Случайно выбранные две колонки:
 [[14  2]
 [ 3 23]
 [22  9]
 [28 24]]


Создайте массив размером $(10, 3)$, который состоит из повторяющихся рядов исходного массива:

In [50]:
data = np.arange(9).reshape(3, 3)
print(data)
import numpy as np

data = np.arange(9).reshape(3, 3)
print("Исходный массив:\n", data)

result = np.tile(data, (4, 1))
result = result[:10, :]

print("Массив из повторяющихся рядов:\n", result)


[[0 1 2]
 [3 4 5]
 [6 7 8]]
Исходный массив:
 [[0 1 2]
 [3 4 5]
 [6 7 8]]
Массив из повторяющихся рядов:
 [[0 1 2]
 [3 4 5]
 [6 7 8]
 [0 1 2]
 [3 4 5]
 [6 7 8]
 [0 1 2]
 [3 4 5]
 [6 7 8]
 [0 1 2]]


Соберите массив, состоящий из элементов 2-й и 4-й колонок и строк с 1-й по 3-ю (**цифры указаны с учётом индексации с 0-ля!**):

In [56]:

import numpy as np

data = np.array([[20, 22, 2, 14, 25, 8],
                [ 7, 25, 23, 3, 22, 15],
                [ 8, 16, 9, 22, 0, 9],
                [ 14, 9, 33, 21, 7, 6],
                [ 4, 24, 24, 28, 3, 40]])
print("Исходная матрица:\n", data)


result = data[1:4, [1, 3]]

print("Новый массив:\n", result)



Исходная матрица:
 [[20 22  2 14 25  8]
 [ 7 25 23  3 22 15]
 [ 8 16  9 22  0  9]
 [14  9 33 21  7  6]
 [ 4 24 24 28  3 40]]
Новый массив:
 [[25  3]
 [16 22]
 [ 9 21]]


Инвертируйте порядок элементов в последних двух колонках:

In [55]:
data = np.array([[20, 22,  2, 14, 25,  8],
                [ 7, 25, 23,  3, 22, 15],
                [ 8, 16,  9, 22,  0,  9],
                [ 14, 9,  33, 21,  7,  6],
                [ 4, 24, 24, 28,  3, 40]])
print(data)
import numpy as np

data = np.array([[20, 22, 2, 14, 25, 8],
                [ 7, 25, 23, 3, 22, 15],
                [ 8, 16, 9, 22, 0, 9],
                [ 14, 9, 33, 21, 7, 6],
                [ 4, 24, 24, 28, 3, 40]])
print("Исходная матрица:\n", data)

data[:, -2:] = data[:, -2:][:, ::-1]

print("Модифицированная матрица:\n", data)




[[20 22  2 14 25  8]
 [ 7 25 23  3 22 15]
 [ 8 16  9 22  0  9]
 [14  9 33 21  7  6]
 [ 4 24 24 28  3 40]]
Исходная матрица:
 [[20 22  2 14 25  8]
 [ 7 25 23  3 22 15]
 [ 8 16  9 22  0  9]
 [14  9 33 21  7  6]
 [ 4 24 24 28  3 40]]
Модифицированная матрица:
 [[20 22  2 14  8 25]
 [ 7 25 23  3 15 22]
 [ 8 16  9 22  9  0]
 [14  9 33 21  6  7]
 [ 4 24 24 28 40  3]]


## Полезные ссылки
* [More than 15 jupyter notebooks to learn Numpy](https://www.kaggle.com/getting-started/115421)
